Carlos Bravo Garrán - 100474964

 # __Análisis Exploratorio de Datos (EDA)__ 

 En este notebook se realizará un Análisis Exploratorio de Datos (EDA) simplificado del conjunto de datos proporcionado, cuyo objetivo es analizar y comprender los factores que influyen en el abandono laboral (Attrition) en una organización.

### 1. __Cargar librerías y datos__


In [ ]:
# Data analysis and wrangling
import numpy as np
import pandas as pd

# Graphs
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns

# Machine learning
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import OneHotEncoder, RobustScaler, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn import tree
from sklearn.metrics import (
    accuracy_score, balanced_accuracy_score, recall_score, roc_curve, auc, confusion_matrix, classification_report
)
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import randint

# Warnings configuration
import warnings
warnings.filterwarnings('ignore')


Cargar el conjunto de datos desde el archivo CSV


In [ ]:
df = pd.read_csv("../data/attrition_availabledata_03.csv")

### 2. __Exploración inicial__

Revisar la estructura general del dataset


In [ ]:
print(df.info())

In [ ]:
dataset_shape = df.shape
print(f"El dataset contiene {dataset_shape[0]} filas y {dataset_shape[1]} columnas.")

In [ ]:
df.describe()

In [ ]:
df.head()

In [ ]:
df[['Attrition']].head()

Este es un problema de __clasificación__, ya que la variable objetivo (Attrition) es binaria (Yes / No). Esto significa que el modelo debe predecir si un empleado abandonará o no la empresa, en lugar de predecir un valor numérico.

### 3. __Identificamos las variables categóricas y numéricas__

In [ ]:
categorical_columns = df.select_dtypes(include=['object']).columns.tolist()
numerical_columns = df.select_dtypes(include=['number']).columns.tolist()
print("Variables categóricas:", categorical_columns)
print("Variables numéricas:", numerical_columns)

#### 3.1 Reclasificar las variables añadiendo ordinales

In [ ]:
categorical_columns = df.select_dtypes(include=['object']).columns.tolist()
ordinal_columns = ["Education", "JobLevel", "EnvironmentSatisfaction", "JobSatisfaction", "WorkLifeBalance", "PerformanceRating", "StockOptionLevel"]
numerical_columns = df.select_dtypes(include=['number']).columns.tolist()

# Eliminamos de numéricas las que hemos clasificado como ordinales
numerical_columns = [col for col in numerical_columns if col not in ordinal_columns]

print("Variables categóricas:", categorical_columns)
print("Variables ordinales:", ordinal_columns)
print("Variables numéricas:", numerical_columns)


#### 3.2 Detectar variables categóricas con alta cardinalidad

 Identificar variables categóricas que pueden generar demasiadas columnas al codificarlas


In [ ]:
categorical_cardinality = df[categorical_columns].nunique().sort_values(ascending=False)
display(categorical_cardinality)


No se considera que existan variables categóricas de alta cardinalidad, por lo tanto no se necesitará realizar ninguna agrupación adicional o una diferente codificación.

### 4. __Análisis de la variable objetivo__

Revisar la distribución de la variable objetivo para identificar desbalance de clases

In [ ]:
if "Attrition" in df.columns:
    plt.figure(figsize=(4,4))
    sns.countplot(x=df["Attrition"], palette="viridis")
    plt.title("Distribución de la variable objetivo (Attrition)\n")
    plt.show()
    
    attrition_counts = df["Attrition"].value_counts(normalize=True)
    display(pd.DataFrame(attrition_counts).rename(columns={"Attrition": "Proportion"}).reset_index(drop=True)*100)


In [ ]:
df.Attrition.value_counts().sort_index().to_frame()

 El dataset está desbalanceado, con 2466 empleados que no abandonan la empresa (NO) y 474 que sí lo hacen (SI). 
 
 Esto significa que la mayoría de los empleados no abandonan la empresa, lo que podría causar que un modelo mal entrenado siempre prediga "No", logrando una precisión aparente alta, pero sin realmente capturar los casos de abandono.



### 5. __Identificar valores nulos__

In [ ]:
missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0].to_frame().reset_index()
missing_values.columns = ["Column Name", "Missing Values"]

display(missing_values)

Ya que los valores faltantes son muy pocos en comparación con el número de datos totales vamos a tomar las siguientes medidas:
- Imputar valores faltantes en las variables numéricas con la mediana
- Imputar valores faltantes en las variables categóricas con la moda

In [ ]:
# Columnas diferenciadas por tipo
num_cols = [col for col in df.select_dtypes(include=['number']).columns if col not in ordinal_columns]
ord_cols = ordinal_columns
cat_cols = df.select_dtypes(include=['object']).columns.tolist()

### 6. __Identificar variables constantes e identidicativas__

#### 6.1 Comprobar si existen columnas con valores únicos

In [ ]:
unique_values = df.nunique()

constant_columns = df.nunique()[df.nunique() == 1].to_frame().reset_index()
constant_columns.columns = ["Column Name", "Unique Value Count"]
constant_columns["Unique Value"] = constant_columns["Column Name"].apply(lambda col: df[col].unique()[0])
constant_columns_list = constant_columns["Column Name"].tolist()

display(constant_columns)

#### 6.2 Comprobar si hay columnas con variables ID

In [ ]:
print(f"Número total de filas: {len(df)}")

unique_values = df.nunique()
print("Número de valores únicos por columna:")
print(unique_values, "\n")

id_columns = [col for col in df.columns if df[col].nunique() == len(df)]
print("Columnas identificativas detectadas:", id_columns)

Como su nombre indica, _EmployeeID_ es una variable identificativa, por lo que es redundante para el estudio. 

Por otro lado, la columna _hours_ tiene 2939 valores, 1 menos que el total de filas. Se podría pensar que es identificativa, pero comprobando los valores vemos que simplemente son valores decimales unos distintos de otros.

#### 6.3 Eliminar las columnas constantes e identificativas

In [ ]:
df = df.drop(columns=constant_columns_list + id_columns, errors='ignore')
numerical_columns = df.select_dtypes(include=['number']).columns.tolist()


print('Se han eliminado las columnas:', constant_columns_list + id_columns)

### 7. __Crear matriz de correlación__

Generar la matriz de correlación para entender relaciones entre las variables numéricas, habiendo eliminado ya las constantes y las identificativas.

In [ ]:
df_numeric = df.select_dtypes(include=['number'])

correlation_matrix = df_numeric.corr()


plt.figure(figsize=(18, 18))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", linewidths=0.5, cmap="coolwarm")
plt.xticks(rotation=75)
plt.title("Matriz de Correlación\n")
plt.show()

No hay correlaciones extremadamente altas, cercanas a 1 o -1, las variables no son redundantes ni son fuertemente dependientes entre sí. Pero sí algunas correlaciones moderadas que podemos considerar.

__Relación entre antigüedad y experiencia laboral:__

- _YearsAtCompany_ y _YearsWithCurrManager_ (0.76): Los empleados que llevan más tiempo en la empresa, más probabilidad de que hayan estado más tiempo con el mismo gerente.
- _YearsAtCompany_ y _TotalWorkingYears_ (0.62): Cuanto más tiempo ha trabajado una persona en general, más tiempo puede haber pasado en la empresa actual.

__Relación entre _PercentSalaryHike_ y _PerformanceRating_ (0.78):__ Hay una alta correlación entre el incremento salarial y la calificación de desempeño. Los empleados con mejor rendimiento reciben mayores aumentos salariales.

- Se podría evaluar si _PercentSalaryHike_ es redundante, ya que está fuertemente ligada a _PerformanceRating_.



En cuanto al resto, tienen correlaciones muy bajas con todas las demás, lo que indica que pueden ser independientes o estar influenciadas por otros factores no considerados. Así, se podría revisar si estas variables tienen algún impacto significativo en la variable objetivo, o si pueden eliminarse.

### 8. __Identificar la correlación con la variable objetivo__

In [ ]:
# Convertir la variable objetivo a numérica
df_aux = df.copy()
df_aux["Attrition"] = df_aux["Attrition"].map({"Yes": 1, "No": 0})

# Asegurar que solo trabajamos con columnas numéricas
df_corr = df_aux.select_dtypes(include=['number'])
attrition_correlation = df_corr.corr()["Attrition"].sort_values()

plt.figure(figsize=(10, 6))
ax = sns.barplot(y=attrition_correlation.index, x=attrition_correlation.values, palette="coolwarm")

# Añadir valores en las barras
for index, value in enumerate(attrition_correlation.values):
    ax.text(value, index, f"{value:.2f}", ha="left", va="center", fontsize=10, color="black")

plt.title("Correlación de 'Attrition' con el resto de variables")
plt.xlabel("Correlación")
plt.ylabel("Variables")
plt.show()


Se puede comprobar que no hay una variable con una correlación extremadamente fuerte con Attrition, pero sí que muchas de ellas tienen una relación muy baja, por ello cabría la posibilidad de considerar su elimiación con el fin de simplificar el modelo.

Junto con las horas trabajadas, las variables de antigüedad (_YearsAtCompany_, _TotalWorkingYears_) son las que más influyen en la retención, se pueden evaluar las relaciones con _YearsWithCurrManager_ para comprobar si esta última sería redundante.

- Se podrían eliminar las variables que tienen muy baja correlación con la objetivo.

In [ ]:
# Filtrar variables con correlación menor a 0.05 en valor absoluto
low_corr_columns = attrition_correlation[abs(attrition_correlation) < 0.05].index.tolist()

# Eliminar estas columnas del dataset
df_filtered = df.drop(columns=low_corr_columns, errors='ignore')

print(f"Se han eliminado en un dataframe de prueba las siguientes columnas por baja correlación: {low_corr_columns}")


### __9. Visualizar relaciones entre las variables correlacionadas__
Realizar una exploración visual de las relaciones entre las variables con alta correlación, para ayudar a decidir si hay redundancias o si algunas variables se deben transformar antes de usarlas en un modelo predictivo.

In [ ]:

# Visualizar relaciones entre variables de antigüedad con pairplot
sns.pairplot(df, vars=["YearsAtCompany", "YearsWithCurrManager", "TotalWorkingYears"], diag_kind="kde")
plt.suptitle("Relaciones entre Antigüedad y Experiencia Laboral", y=1.02)
plt.show()



__Relación entre Antigüedad y Experiencia Laboral__

- _YearsAtCompany_ y _YearsWithCurrManager_ (0.76): Relación fuerte, posible redundancia.
- _YearsAtCompany_ y _TotalWorkingYears_ (0.62): Relación esperada, pero aporta información diferente.
- Conclusión: _YearsAtCompany_ o _YearsWithCurrManager_ podría ser redundante.

Se podría evaluar impacto en el modelo y eliminar una si es necesario.

Habiendo identificado y evaluado las variables, comprobando valores nulos, valores constantes e identificativos y las correlaciones entre ellas, se puede pasar a la evaluación de los posibles modelos de clasificación.

## __Evaluación de Modelos de Clasificación con Preprocesamiento Avanzado__
En esta parte se implementa un pipeline de preprocesamiento y modelado para predecir la variable Attrition en un dataset de empleados. Se va a utilizar validación cruzada para la evaluación interna (inner evaluation) y una evaluación final con un conjunto de prueba independiente (outer evaluation).

Los principales pasos seguidos son:

1. División de datos en conjuntos de entrenamiento y prueba (2/3 para entrenar el modelo - 1/3 para evaluar el rendimiento final).
2. Preprocesamiento de datos, incluyendo imputación, escalado, codificación y reducción de dimensionalidad.
3. Evaluación interna (inner evaluation) mediante validación cruzada estratificada.
4. Entrenamiento y evaluación final (outer evaluation) con métricas clave como balanced accuracy, accuracy, TPR, TNR y matriz de confusión.

### __1. División de Datos en Train y Test__

Se separan las variables predictoras (X) de la variable objetivo (y) y realiza la división de los datos en conjuntos de entrenamiento y prueba

In [ ]:
X = df.drop(columns=["Attrition"])
y = df["Attrition"]

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=1/3, random_state=100474964)

### __2. Preprocesamiento de Datos__
El preprocesamiento se divide en tres tipos de variables:

- Variables numéricas: Se imputan con KNNImputer y se escalan con RobustScaler.
- Variables categóricas: Se imputan con la moda (most_frequent) y se codifican con OneHotEncoder, seguido de una reducción de dimensionalidad con PCA(n_components=5).
- Variables ordinales: Se imputan con la mediana y se transforman con OrdinalEncoder.


#### 2.1 Identificación de Tipos de Variables


In [ ]:
numerical_columns = X.select_dtypes(include=['number']).columns.tolist()
categorical_columns = X.select_dtypes(include=['object']).columns.tolist()
ordinal_columns = ["Education", "JobLevel", "EnvironmentSatisfaction", "JobSatisfaction", "WorkLifeBalance", "PerformanceRating", "StockOptionLevel"]


#### 2.2 Transformaciones por Tipo de Variable

In [ ]:

# Aplicar Label Encoding a variables ordinales
ord_encoder = OrdinalEncoder()
df[ordinal_columns] = ord_encoder.fit_transform(df[ordinal_columns])

ord_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
])

# Pipeline para datos categóricos
cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False)),
    ('pca', PCA(n_components=5))
])


# Pipeline para datos numéricos
num_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=5)),
    ('scaler', RobustScaler())
])

# ColumnTransformer con todos los preprocesamientos
preprocessor = ColumnTransformer(transformers=[
    ('num', num_transformer, numerical_columns),
    ('cat', cat_transformer, categorical_columns),
    ('ord', ord_transformer, ordinal_columns)
])

preprocessor

### __3. Conversión de Variable Objetivo (Attrition)__
Convertimos la variable Attrition a un formato binario (1 para "Yes" y 0 para "No")

In [ ]:
y_train = y_train.map({"Yes": 1, "No": 0})
y_test = y_test.map({"Yes": 1, "No": 0})


### __4. Aplicación del Preprocesador__
El preprocesador se ajusta y transforma los datos de entrenamiento y prueba

In [ ]:
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)


In [ ]:
def evaluate_model(model, X_test, y_test, model_name):
    """
    Evaluates a classification model and displays key metrics, confusion matrix, and ROC curve.
    """
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else None
    
    acc = accuracy_score(y_test, y_pred)
    bal_acc = balanced_accuracy_score(y_test, y_pred)
    tpr = recall_score(y_test, y_pred)  # Sensitivity / TPR
    tnr = recall_score(y_test, y_pred, pos_label=0)  # Specificity / TNR
    conf_matrix = confusion_matrix(y_test, y_pred)

    print(f"=== Final Evaluation: {model_name} ===")
    print(f"Accuracy: {acc:.4f}")
    print(f"Balanced Accuracy: {bal_acc:.4f}")
    print(f"TPR (Sensitivity): {tpr:.4f}")
    print(f"TNR (Specificity): {tnr:.4f}")
    print("\nConfusion Matrix:")
    print(classification_report(y_test, y_pred))

    plt.figure(figsize=(4, 4))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", cbar=False)
    plt.xlabel("Predicted Labels")
    plt.ylabel("True Labels")
    plt.title(f"Confusion Matrix ({model_name})")
    plt.show()

    if y_prob is not None:
        fpr, tpr_curve, _ = roc_curve(y_test, y_prob)
        auc_score = auc(fpr, tpr_curve)

        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr_curve, label=f'{model_name} (AUC = {auc_score:.4f})', color='blue')
        plt.plot([0, 1], [0, 1], linestyle='--', color='gray')  
        
        plt.xlabel("False Positive Rate (FPR)")
        plt.ylabel("True Positive Rate (TPR)")
        plt.title(f"ROC Curve for {model_name}")
        plt.legend()
        plt.grid()
        plt.show()
    
        return {"Accuracy": acc, "Balanced Accuracy": bal_acc, "TPR": tpr, "TNR": tnr, "AUC": auc_score}
    
    return {"Accuracy": acc, "Balanced Accuracy": bal_acc, "TPR": tpr, "TNR": tnr, "AUC": None}

### __5. Evaluación con Modelo Dummy__

Se entrena un modelo Dummy para establecer como puntos de referencia:

- Balanced Accuracy esperada si el modelo fuera trivial.
- Comparación con los modelos entrenados para demostrar su efectividad.

Así se podrá verificar si los modelos realmente aprenden patrones o simplemente reflejan la distribución de clases.

In [ ]:
from sklearn.dummy import DummyClassifier

dummy = DummyClassifier(strategy="most_frequent")
dummy.fit(X_train_transformed, y_train)
dummy_ev = evaluate_model(dummy, X_test_transformed, y_test, "Modelo Dummy")


### __6. Definición de los Modelos con Pipeline__
Se construye un pipeline que incluye el preprocesamiento y el modelo de clasificación (Regresión Logística con balanceo de clases)

In [ ]:

clf_tree = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', tree.DecisionTreeClassifier(class_weight='balanced'))
])

clf_tree

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

clf_knn = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', KNeighborsClassifier(n_neighbors=5))
])

clf_knn

### __7. Evaluación Interna (Inner Evaluation)__
Para la evaluación interna, se utiliza validación cruzada estratificada (StratifiedKFold) con 5 divisiones (n_splits=5)

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=100474964)
cross_val_scores = cross_val_score(clf_tree, X_train, y_train, cv=skf, scoring='balanced_accuracy')
print(f"Balanced Accuracy (inner evaluation): {np.mean(cross_val_scores):.4f}")

### __8. Entrenamiento del Modelo__
El modelo se ajusta con todo el conjunto de entrenamiento

In [ ]:
clf_tree.fit(X_train, y_train)
clf_knn.fit(X_train, y_train)

### __9. Evaluación con Test (Outer Evaluation)__
Se realiza la predicción sobre el conjunto de prueba y se calculan las métricas principales:

- Balanced Accuracy: Promedio de TPR y TNR.
- Accuracy: Proporción de predicciones correctas.
- TPR (Sensibilidad/Recall): Qué tan bien el modelo detecta los casos positivos.
- TNR (Especificidad): Qué tan bien el modelo detecta los casos negativos.
- Matriz de Confusión: Visualización detallada de aciertos y errores.

In [ ]:
d_tree_ev = evaluate_model(clf_tree, X_test, y_test, "Árbol de Decisión (Base)")
knn_ev = evaluate_model(clf_knn, X_test, y_test, "KNN (Base)")


### __10. Optimización de Modelos con RandomizedSearchCV__

Se usa búsqueda aleatoria de hiperparámetros (`RandomizedSearchCV`) para encontrar la mejor configuración de los modelos con el fin de mejorar el rendimiento del modelo ajustando sus hiperparámetros de manera eficiente.

- KNN (`n_neighbors`, `weights`, `metric`).  
- Árbol de Decisión (`max_depth`, `criterion`).  


In [ ]:
param_dist_knn = {"n_neighbors": randint(3, 20), "weights": ["uniform", "distance"], "metric": ["euclidean", "manhattan"]}
param_dist_tree = {"max_depth": randint(3, 20), "criterion": ["gini", "entropy"]}

random_knn = RandomizedSearchCV(KNeighborsClassifier(), param_distributions=param_dist_knn, n_iter=10, cv=5, scoring="balanced_accuracy", n_jobs=-1, random_state=42)
random_tree = RandomizedSearchCV(DecisionTreeClassifier(), param_distributions=param_dist_tree, n_iter=10, cv=5, scoring="balanced_accuracy", n_jobs=-1, random_state=42)

random_knn.fit(X_train_transformed, y_train)
random_tree.fit(X_train_transformed, y_train)

print(f"Mejores hiperparámetros para KNN: {random_knn.best_params_}")
print(f"Balanced Accuracy KNN: {random_knn.best_score_:.4f}")
print(f"Mejores hiperparámetros para Árbol de Decisión: {random_tree.best_params_}")
print(f"Balanced Accuracy Árbol de Decisión: {random_tree.best_score_:.4f}")



### __11. Evaluación de los Modelos Optimizados__

Para confirmar que la optimización de hiperparámetros realmente mejora el rendimiento.
- Se seleccionan los mejores hiperparámetros y se entrenan los modelos definitivos.  
- Se calculan nuevamente todas las métricas para ver la mejora con respecto a los modelos por defecto.



In [ ]:
opt_tree_ev = evaluate_model(random_tree.best_estimator_, X_test_transformed, y_test, "Árbol de Decisión (Optimizado)")
opt_knn_ev = evaluate_model(random_knn.best_estimator_, X_test_transformed, y_test, "KNN (Optimizado)")